In [1]:
import pandas as pd
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import requests
from datetime import datetime
from datetime import timedelta
import time
import json

from pymongo.errors import OperationFailure, DuplicateKeyError

pd.set_option('precision', 2)
pd.set_option('max_rows', 20)
pd.set_option('max_colwidth', 30)
# pd.describe_option('max_rows')
# pd.describe_option('precision')
# pd.describe_option('max_colwidth')

### Connect, dbs

In [2]:
client = MongoClient(host="localhost",port=27017,username ="root", password="example")
database = client["bdl03"]
collection_submissions = database['submissions']

### Collections


In [3]:
# # Get Bulk load of Reddit submissions


# ## 1. Create method for api call

def get_pushshift_data(data_type, **kwargs):
    """Get data from pushshift api -> Reddit
    param p1: String -> "comment" or "submission"
    param p2: Object -> parameters defined in 2.
    return: json file
    """
    base_url = f"https://api.pushshift.io/reddit/search/{data_type}/"
    payload = kwargs
    request = requests.get(base_url, params=payload)
    if request.status_code == 200:
        return request.json()
    else:
        return {}


# ## 2. Define parameters for api call
path = "C:/Users/annam/OneDrive/Documents/dev/MSC/BDL1/Semesterproject"
data_type = "submission"
query="Netflix"
size=500 #500 is max
sort_type="created_utc"
sort="desc"
aggs="created_utc"
fields = ["subreddit",
         "subreddit_id",
         "title",
         "created_utc",
         "permalink",
         "id",
         "num_comments",
         "num_crossposts",
         "score",
         "upvote_ratio"]

subreddits = ["Netflix"]#,"WallStreetBets","Piracy","Movies","Television"]
for subreddit in subreddits:

    # bulk_df = pd.DataFrame()
    current_timestamp = time.time()
    dynamic_timestamp = current_timestamp - timedelta(days=1) #(60 * 60 * 24 * 730) #two years ago
    while int(dynamic_timestamp) < int(current_timestamp):
        data = get_pushshift_data(data_type=data_type,q=query,after=int(dynamic_timestamp),sort_type="created_utc",sort="Asc",
                                      fields=fields,size=size,subreddit=subreddit)
        # df = pd.json_normalize(data['data'])
        df = data["data"]
        if len(df) > 0:
            dynamic_timestamp = max(df["created_utc"].values) + 1
            collection_submissions.insert_many(df)
            # bulk_df = bulk_df.append(df)
        else:
            print("done...")
            break;

    #parse date
    # bulk_df["created_utc"] = pd.to_datetime(bulk_df["created_utc"], unit="s", errors='ignore')


    #get latest date for file name #why do i need to convert it again?!
    # latestDate = pd.to_datetime(max(bulk_df["created_utc"]))
    #write to csv
    # bulk_df.to_csv(path+"/submissions/Reddit_BULK_{}_Netflix_{}.csv".format(subreddit,latestDate.strftime('%Y%m%d')), sep=",", index=False)

TypeError: unsupported operand type(s) for -: 'float' and 'datetime.timedelta'

In [4]:
subreddits = ["WallStreetBets"]
for subreddit in subreddits:

    # bulk_df = pd.DataFrame()
    current_timestamp = time.time()
    dynamic_timestamp = current_timestamp - (60 * 60 * 24 * 730) #two years ago #(60*60*24)#
    while int(dynamic_timestamp) < int(current_timestamp):
        data = get_pushshift_data(data_type=data_type,q=query,after=int(dynamic_timestamp),sort_type="created_utc",sort="Asc",
                                      fields=fields,size=size,subreddit=subreddit)
        # df = pd.json_normalize(data['data'])
        df = data["data"]
        if len(df) > 0:
            dynamic_timestamp = max([x['created_utc'] for x in df]) + 1
            submission_ids = []
            for element in df:
                try:
                    json_el = json.loads(json.dumps(element))
                    database.submissions.insert_one(json_el)
                    print(f"insertet submission {json_el['id']}")
                    submission_ids.append(json_el['id'])

                except(DuplicateKeyError):
                    print('Duplicate submission caught')

                for submission_id in submission_ids:
                    comment_ids_data = get_submission_comment_ids(submission_id)
                    if comment_ids_data != None:
                        for d in comment_ids_data['data']:
                            comments_ids = str(d)
                            comments = get_comments(comments_ids)
                            try:
                                json_comment = json.loads(json.dumps(comments['data'][0]))
                                json_comment["submission_id"] = submission_id
                                json_comment["_id"] = json_comment["id"]
                            except TypeError:
                                print("Comment id {} returned None Comment was {}".format(d, comments))
                            # database.comments.insert_one(json_el)
                            # print(f"insertet comment")
                            try:
                                database.comments.insert_one(json_comment)
                                print(f"insertet comment")
                            except(DuplicateKeyError):
                                print(f'Duplicate comment caught: {DuplicateKeyError}')
                        print(f"Done with comments of submission {submission_id}")



    else:
            print("done...")
            break;

OperationFailure: Authentication failed., full error: {'ok': 0.0, 'errmsg': 'Authentication failed.', 'code': 18, 'codeName': 'AuthenticationFailed'}

In [5]:
for element in data:
    
    try:
        database.Accidents.insert_one(element)
        print(f"insertet accident")
    except(DuplicateKeyError):
        print(f'Duplicate comment caught: {DuplicateKeyError}')

TypeError: document must be an instance of dict, bson.son.SON, bson.raw_bson.RawBSONDocument, or a type that inherits from collections.MutableMapping

In [6]:
with open('C:/Users/annam/OneDrive - Hochschule Luzern/Lehre/TwitterIBR_dbo_LandingZone.json', 'rb') as f:
    j = bigjson.load(f)
    counter = 0
    for i in range(0, 2):  #15241400
        try:
            element = j[int(i)]
        except:
            print(f"Sorry no index {i}")

        try:
            database.test.insert_one(element)
            print("inserted tweet")
        except(DuplicateKeyError):
            print('Duplicate caught')

NameError: name 'bigjson' is not defined

In [7]:
gemeinden_df = pd.read_csv('C:/Users/annam/OneDrive/Documents/dev/MSC/BDL1/Semesterproject/bfs_municipality.csv')   # loading csv file


for index, row in gemeinden_df.iterrows():
    json_row = json.loads(json.dumps(row))
    print(json_row)
    

TypeError: Object of type Series is not JSON serializable

## Join

The collection "gemeinden" is connected to the collection "Accidents" via the field "IntMunicipalityCode" using "$lookup". If you only use this query, you will get the value None when aggregating the field "gemeinde_info". To correct this error, you have to append the following command to the query

<code> {
                '$unwind':'$gemeinde_info'
} </code>

The results of the join per document are returned as an array. Using the "$unwind" command, the objects are extracted from the array. One document is displayed for each object.



In [8]:
###### DOES NOT WORK 
c = database.Accidents.aggregate([
    priceQtyConversionStage,
    

    #Join with gemeinden table
    {
        "$lookup":{
            "from": "gemeinden",       #other table name
            "localField": "IntMunicipalityCode",   #name of users table field
            "foreignField": "IntMunicipalityCode", # name of userinfo table field
            "as": "gemeinde_info"         # alias for userinfo table
        }
    },
    {  "$unwind":"$gemeinde_info" },    # $unwind used for getting data in object or for one record only



    # define which fields are you want to fetch
    {   
        "$project":{
            'AccidentType':1, 'AccidentType_en':1, 'AccidentSeverityCategory':1,
       'AccidentSeverityCategory_en':1, 'AccidentInvolvingPedestrian':1,
       'AccidentInvolvingBicycle':1, 'AccidentInvolvingMotorcycle':1, 'RoadType':1,
       'RoadType_en':1, 'CantonCode':1, 'MunicipalityCode':1, 'AccidentYear':1,
       'AccidentMonth':1, 'AccidentMonth_en':1, 'AccidentWeekDay':1,
       'AccidentWeekDay_en':1, 'AccidentHour':1, 'AccidentHour_text':1, '_id':1,"MunicipalityName": "$gemeinden.GDENAME",
            
        } 
    }
]);

df_total= pd.DataFrame(c)

NameError: name 'priceQtyConversionStage' is not defined